In [1]:
!pip install hyperopt

In [14]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [3]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim==1: y_train = to_categorical(y_train)
if y_test.ndim==1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')

  logdir = os.path.join('logs', datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  model.fit(
      X_train, 
      y_train,
      batch_size=params_fit.get('batch_size', 128),
      epochs=params_fit.get('epochs', 5),
      verbose=params_fit.get('verbose', 1),
      validation_data=params_fit.get('validation_data', (X_train, y_train)),
      callbacks=[tensorboard_callback]
      )
  
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax(y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax(y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(0.3),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(0.3),

    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(0.3),
    Dense(1024, activation='relu'),
    Dropout(0.3),

    Dense(num_classes, activation='softmax')                    
  ])

In [11]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 13s 49ms/step - loss: 2.7305 - accuracy: 0.2650 - val_loss: 1.0649 - val_accuracy: 0.6790
Epoch 2/5
272/272 [==============================] - 13s 46ms/step - loss: 0.8768 - accuracy: 0.7242 - val_loss: 0.2808 - val_accuracy: 0.9175
Epoch 3/5
272/272 [==============================] - 13s 46ms/step - loss: 0.4234 - accuracy: 0.8683 - val_loss: 0.1396 - val_accuracy: 0.9570
Epoch 4/5
272/272 [==============================] - 13s 46ms/step - loss: 0.2691 - accuracy: 0.9175 - val_loss: 0.0584 - val_accuracy: 0.9834
Epoch 5/5
272/272 [==============================] - 13s 46ms/step - loss: 0.1905 - accuracy: 0.9436 - val_loss: 0.0498 - val_accuracy: 0.9850


0.9510204081632653

In [0]:
def get_model(params):
  return Sequential([
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=input_shape),
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', padding='same'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_one']),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_two']),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'),
    Conv2D(filters=128, kernel_size=(3,3), activation='relu'),
    MaxPool2D(),
    Dropout(params['dropout_cnn_block_three']),

    Flatten(),
    
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_one']),
    Dense(1024, activation='relu'),
    Dropout(params['dropout_dense_block_two']),

    Dense(num_classes, activation='softmax')                    
  ])

In [0]:
def func_obj(params):
  model = get_model(params)

  model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='Adam')

  model.fit(
      X_train, 
      y_train,
      batch_size=int(params.get('batch_size', 128)),
      epochs=5,
      verbose=0,
  )
  
  accuracy = model.evaluate(X_test, y_test)[1]
  print(params, f'accuracy={accuracy}')

  return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [22]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.2, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.2, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.2, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.2, 0.5),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.2, 0.5),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    25,
    Trials()
)

  1/138 [..............................]
 - ETA: 0s - loss: 2.7167 - accuracy: 0.3438

  7/138 [>.............................]
 - ETA: 1s - loss: 0.4129 - accuracy: 0.8973

 11/138 [=>............................]
 - ETA: 1s - loss: 0.3506 - accuracy: 0.9062

 16/138 [==>...........................]
 - ETA: 1s - loss: 0.2819 - accuracy: 0.9258

 22/138 [===>..........................]
 - ETA: 1s - loss: 0.5191 - accuracy: 0.9048

 27/138 [====>.........................]
 - ETA: 1s - loss: 0.7120 - accuracy: 0.9039
